# 1. Install and Import Dependencies

In [1]:
!pip install mediapipe opencv-python

You should consider upgrading via the 'c:\users\junyup\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [2]:
import mediapipe as mp
import cv2
import numpy as np
import uuid
import os

In [3]:
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

# 2. Draw Hand Skeletons
<img src=https://i.imgur.com/qpRACer.png />

In [4]:
def get_label(index, hand, results):
    output = None
    for index, classification in enumerate(results.multi_handedness):
        if classification.classification[0].index == index:
            
            # Process results
            label = classification.classification[0].label[0:]
            text = '{}'.format(label)
                    
            output = text
            
    return output

In [5]:
def check_coord(index, hand, results):
    output = None
    for idx, classification in enumerate(results.multi_handedness):
        if classification.classification[0].index == index:
            
            # Process results
            label = classification.classification[0].label[0:]
            text = '{}'.format(label)
                    
            output = text
            
    return output

In [6]:
cap = cv2.VideoCapture(0)

with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5) as hands: 
    while cap.isOpened():
        ret, frame = cap.read()
        
        # BGR 2 RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Flip on horizontal
        image = cv2.flip(image, 1)
        
        # Set flag
        image.flags.writeable = False
        
        # Detections
        results = hands.process(image)
        
        # Set flag to true
        image.flags.writeable = True
        
        # RGB 2 BGR
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Detections
        #print(results)
        
        # Rendering results
        if results.multi_hand_landmarks:
            for num, hand in enumerate(results.multi_hand_landmarks):
                mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS, 
                                        mp_drawing.DrawingSpec(thickness=2, circle_radius=4),
                                        mp_drawing.DrawingSpec(thickness=2, circle_radius=2),
                                         )
                image_height, image_width, _ = image.shape
                center = np.array([(hand.landmark[mp_hands.HandLandmark.WRIST].x 
                                   + hand.landmark[mp_hands.HandLandmark.INDEX_FINGER_MCP].x 
                                   + hand.landmark[mp_hands.HandLandmark.PINKY_MCP].x) /3 * image_width,
                                   (hand.landmark[mp_hands.HandLandmark.WRIST].y 
                                   + hand.landmark[mp_hands.HandLandmark.INDEX_FINGER_MCP].y 
                                   + hand.landmark[mp_hands.HandLandmark.PINKY_MCP].y) /3 * image_height])
                
                wrist = np.array([hand.landmark[mp_hands.HandLandmark.WRIST].x * image_width, 
                              hand.landmark[mp_hands.HandLandmark.WRIST].y * image_height])
                thumb = np.array([hand.landmark[mp_hands.HandLandmark.THUMB_TIP].x * image_width, 
                              hand.landmark[mp_hands.HandLandmark.THUMB_TIP].y * image_height])
                index = np.array([hand.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x * image_width, 
                              hand.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y * image_height])
                middle = np.array([hand.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].x * image_width, 
                              hand.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].y * image_height])
                ring = np.array([hand.landmark[mp_hands.HandLandmark.RING_FINGER_TIP].x * image_width, 
                              hand.landmark[mp_hands.HandLandmark.RING_FINGER_TIP].y * image_height])
                pinky = np.array([hand.landmark[mp_hands.HandLandmark.PINKY_TIP].x * image_width, 
                             hand.landmark[mp_hands.HandLandmark.PINKY_TIP].y * image_height])
                cv2.putText(image, "Thumb "+"{0:0.2f}".format(np.sqrt((thumb[0]-wrist[0])**2+(thumb[1]-wrist[1])**2)), (0,40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
                cv2.putText(image, " Index "+ "{0:0.2f}".format(np.sqrt((index[0]-wrist[0])**2+(index[1]-wrist[1])**2)), (0,60), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
                cv2.putText(image, "Middle "+ "{0:0.2f}".format(np.sqrt((middle[0]-wrist[0])**2+(middle[1]-wrist[1])**2)), (0,80), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
                cv2.putText(image, "  Ring "+ "{0:0.2f}".format(np.sqrt((ring[0]-wrist[0])**2+(ring[1]-wrist[1])**2)), (0,100), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
                cv2.putText(image, " Pinky "+ "{0:0.2f}".format(np.sqrt((pinky[0]-wrist[0])**2+(pinky[1]-wrist[1])**2)), (0,120), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
                
                if get_label(num, hand, results):
                    text = get_label(num, hand, results)
                    cv2.putText(image, text, (0,20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
                hm = hand.landmark
                mp = mp_hands.HandLandmark
                
                max_width = max(hm[mp.WRIST].x - center[0], hm[mp.THUMB_CMC].x - center[0], hm[mp.THUMB_MCP].x - center[0],
                                hm[mp.THUMB_IP].x - center[0], hm[mp.THUMB_TIP].x - center[0], 
                                hm[mp.INDEX_FINGER_MCP].x - center[0], hm[mp.INDEX_FINGER_PIP].x - center[0],
                                hm[mp.INDEX_FINGER_DIP].x - center[0], hm[mp.INDEX_FINGER_TIP].x - center[0],
                                hm[mp.MIDDLE_FINGER_MCP].x - center[0], hm[mp.MIDDLE_FINGER_PIP].x - center[0],
                                hm[mp.MIDDLE_FINGER_DIP].x - center[0], hm[mp.MIDDLE_FINGER_TIP].x - center[0],
                                hm[mp.RING_FINGER_MCP].x - center[0], hm[mp.RING_FINGER_PIP].x - center[0],
                                hm[mp.RING_FINGER_DIP].x - center[0], hm[mp.RING_FINGER_TIP].x - center[0],
                                hm[mp.PINKY_MCP].x - center[0], hm[mp.PINKY_PIP].x - center[0],
                                hm[mp.PINKY_DIP].x - center[0], hm[mp.PINKY_TIP].x - center[0]) *image_width
                max_height = max(hm[mp.WRIST].y - center[1], hm[mp.THUMB_CMC].y - center[1], hm[mp.THUMB_MCP].y - center[1],
                                hm[mp.THUMB_IP].y - center[1], hm[mp.THUMB_TIP].y - center[1], 
                                hm[mp.INDEX_FINGER_MCP].y - center[1], hm[mp.INDEX_FINGER_PIP].y - center[1],
                                hm[mp.INDEX_FINGER_DIP].y - center[1], hm[mp.INDEX_FINGER_TIP].y - center[1],
                                hm[mp.MIDDLE_FINGER_MCP].y - center[1], hm[mp.MIDDLE_FINGER_PIP].y - center[1],
                                hm[mp.MIDDLE_FINGER_DIP].y - center[1], hm[mp.MIDDLE_FINGER_TIP].y - center[1],
                                hm[mp.RING_FINGER_MCP].y - center[1], hm[mp.RING_FINGER_PIP].y - center[1],
                                hm[mp.RING_FINGER_DIP].y - center[1], hm[mp.RING_FINGER_TIP].y - center[1],
                                hm[mp.PINKY_MCP].y - center[1], hm[mp.PINKY_PIP].y - center[1],
                                hm[mp.PINKY_DIP].y - center[1], hm[mp.PINKY_TIP].y - center[1]) *image_height
                
                
                cv2.circle(image, (int(center[0]), int(center[1])), 2, (255, 255, 255), -1)
                lu = (center[0] - (max_width/2), center[1] - (max_height/2))
                rd = (center[0] + (max_width/2), center[1] + (max_height/2))
                cv2.rectangle(image, (int(lu[0]), int(lu[1])), (int(rd[0]), int(rd[1])), (255, 255, 255), 1)

        cv2.imshow('Hand Tracking (Press `q` to Quit)', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

In [7]:
mp_hands.HandLandmark.THUMB_TIP

<HandLandmark.THUMB_TIP: 4>